# Evaluación Modular:
# Optimización de Modelos para la Predicción de Enfermedades Crónicas

## Objetivo
Desarrollar y optimizar un modelo de aprendizaje automático capaz de predecir la probabilidad de que un paciente desarrolle una enfermedad crónica específica, utilizando técnicas avanzadas de ajuste de hiperparámetros con Optuna y Ray Tune. Se espera que el modelo optimizado proporcione predicciones precisas y generalizables, contribuyendo a la identificación temprana y prevención de enfermedades crónicas en el ámbito de la salud pública.

**Dataset utilizado:**  
[Disease Prediction Using Machine Learning](https://www.kaggle.com/datasets/kaushil268/disease-prediction-using-machine-learning/discussion)

---

### Estructura del Notebook:
1. Metodología.
2. Configuración inicial del notebook.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

---

### Flujo de trabajo
1. **Carga de datos:**
   - Carga de datos, se concatenan los archivos **Training.csv** y **Testing.csv** en una solo data.
   - Escalado de características con `StandardScaler`.
   - División de datos en entrenamiento y prueba (70% / 30%) con estratificación de clases.

2. **Modelos creados:**
   - **Modelo base:** RandomForestClassifier con hiperparámetros por defecto.
   - **Modelo optimizado con Optuna:** RandomForestClassifier optimizado mediante Optuna con 20 trials.
   - **Modelo optimizado con Ray Tune:** RandomForestClassifier optimizado mediante Ray Tune con 20 trials. Uso de CallBack para mostrar logs.

3. **Evaluación de modelos:**
   - **Métricas evaluadas:**
      - F1 Score.
      - Tiempo de ejecución.

4. **Comparación y visualización:**
   - Resumen del dataset.
   - Mejores hiperparametros encontrados por Optuna y Ray Tune.
   - Resultados del modelo base, optimizado con Optuna y optimizado con Ray Tune.
   - Gráficos de comparación de rendimiento (F1-Score) y tiempos de ejecución.
   - Gráficos de evolución de F1-Score en los modelos optimizados con Optuna y Ray Tune.

---

# 2. Configuración inicial del notebook
- Importación de librerias necesarias.

--- 

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import optuna
import ray
from ray import train, tune
from ray.tune import Tuner
from ray.tune.search.basic_variant import BasicVariantGenerator
from ray.tune import Callback
from datetime import datetime

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 0:** Configuración del entorno

- **`setup_environment()`** 
Debido a lso multiples errores y warnings que ocurren durante la ejecución, se decidió configurar el setup para mejorar la legibilidad de las salidas de codigo.

In [ ]:
def setup_environment():
    """
    Configura variables de entorno, logging y estilo gráfico.
    Esta función se debe llamar al inicio del flujo principal.
    """

    # ===== Desactiva servicios externos de Apache Arrow que no se usarán =====
    os.environ['ARROW_DISABLE_S3'] = '1'       # Desactiva soporte para Amazon S3 (ahorra memoria si no se usa)
    os.environ['ARROW_DISABLE_GCS'] = '1'      # Desactiva soporte para Google Cloud Storage
    os.environ['ARROW_DISABLE_HDFS'] = '1'     # Desactiva soporte para Hadoop File System

    # ===== Configura Ray para entornos locales (como notebooks) =====
    os.environ['RAY_DISABLE_DASHBOARD'] = '1'  # Evita que Ray inicie su dashboard web (consume recursos)
    os.environ['RAY_ENABLE_WINDOWS_OR_OSX_CLUSTER'] = '1'  # Permite ejecutar Ray en Windows/macOS
    os.environ['RAY_LOG_TO_STDERR'] = '1'      # Redirige los logs de Ray a stderr (consola)
    os.environ['RAY_DEDUP_LOGS'] = '0'         # Muestra todos los logs, incluso si están repetidos

    # ===== Reduce verbosidad de logs de Apache Arrow =====
    os.environ['ARROW_VERBOSE_THRESHOLD'] = '0'  # Reduce al mínimo los mensajes de log de Arrow

    # ===== Configura el logging de Python (global) =====
    logging.basicConfig(level=logging.ERROR)     # Solo se mostrarán errores (oculta warnings/info)

    # ===== Configura el estilo visual de los gráficos =====
    sns.set_style("whitegrid")                   # Estilo de seaborn con fondo blanco y cuadrículas
    plt.rcParams['figure.figsize'] = (12, 6)     # Tamaño por defecto de las figuras (ancho x alto)
    plt.rcParams['font.size'] = 12               # Tamaño de fuente legible
    plt.rcParams['figure.max_open_warning'] = 50 # Permite abrir hasta 50 figuras sin warning

**Bloque 1:** Carga y preprocesamiento de datos

- **`load_and_preprocess_data()`** 
Carga el dataset de diagnóstico de enfermedades, divide los datos en conjuntos de entrenamiento y prueba, codifica etiquetas, escala los datos y genera un resumen del dataset.

In [ ]:
from sklearn.preprocessing import LabelEncoder#, label_binarize
def load_and_preprocess_data(data):
    """
    Realiza la carga y el preprocesamiento del dataset de cáncer de mama.

    Este paso incluye:
    - División del conjunto de datos en entrenamiento y prueba (70/30, con estratificación).
    - Escalamiento de características numéricas usando StandardScaler.
    - Creación de un resumen informativo del dataset.

    Args:
        data (sklearn.utils.Bunch): Objeto cargado con `load_breast_cancer()` de scikit-learn.

    Returns:
        tuple:
            - X_train_scaled (np.ndarray): Datos de entrenamiento escalados.
            - X_test_scaled (np.ndarray): Datos de prueba escalados.
            - y_train (np.ndarray): Etiquetas de entrenamiento.
            - y_test (np.ndarray): Etiquetas de prueba.
            - data (Bunch): El dataset original sin modificar.
            - info_df (pd.DataFrame): DataFrame resumen del dataset (tamaño, clases, particiones, etc.).
    """

    # Separar características y variable objetivo
    X = data.drop(columns=['prognosis'])
    y = data['prognosis']
    
    # División train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Codificar etiquetas
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)

    # Escalamiento
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Obtener clases y distribución
    clases = sorted(y.unique())  # Ej: ['asma', 'bronquitis', 'lupus', ...]
    clases_str = ", ".join(str(c) for c in clases)

    # Distribución de clases
    counts = y.value_counts()  # Series con índices = clases, valores = conteo
    distribucion_clases = ", ".join([f"{v} ({k})" for k, v in counts.items()])
    
    # Resumen
    info_dict = {
        "Total muestras": [X.shape[0]],
        "N° características": [X.shape[1]],
        "N° clases": [len(clases)],
        "Clases": [clases_str],
        "Distribución clases": [distribucion_clases],
        "Train size": [X_train.shape[0]],
        "Test size": [X_test.shape[0]],
        "Shape X_train": [X_train_scaled.shape],
        "Shape X_test": [X_test_scaled.shape]
    }
    info_df = pd.DataFrame(info_dict)
    
    return X_train_scaled, X_test_scaled, y_train_enc, y_test_enc, data, info_df

**Bloque 2:** Aplicación del modelo base, y optimizados con Optuna y Ray Tune

- **`train_baseline_model()`** 
Entrena y evalúa un modelo base Random Forest con hiperparámetros por defecto.

- **`optimize_with_optuna()`** 
Realiza optimización de hiperparámetros para un modelo Random Forest usando Optuna.

- **`optimize_with_ray_tune()`** 
Realiza optimización de hiperparámetros para un modelo Random Forest utilizando Ray Tune

In [ ]:
def train_baseline_model(X_train, y_train, X_test, y_test, target_names):
    """
    Entrena y evalúa un modelo base Random Forest con hiperparámetros por defecto.

    El modelo se ajusta sobre los datos de entrenamiento y se evalúa usando F1-score
    sobre el conjunto de prueba.

    Args:
        X_train (np.ndarray): Conjunto de entrenamiento (features) escalado.
        y_train (np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray): Conjunto de prueba (features) escalado.
        y_test (np.ndarray): Etiquetas del conjunto de prueba.
        target_names (np.ndarray): Nombres de las clases objetivo (no usado directamente aquí, 
                                   pero útil para extensiones o reportes detallados).

    Returns:
        tuple:
            - base_model (RandomForestClassifier): Modelo entrenado.
            - base_f1 (float): F1-score en el conjunto de prueba.
            - base_train_time (float): Tiempo de entrenamiento (segundos).
            - y_pred_base (np.ndarray): Predicciones del modelo base en el test set.
    """
    # Entrenar modelo con parámetros por defecto
    start_time = time.time()
    base_model = RandomForestClassifier(random_state=42)
    base_model.fit(X_train, y_train)
    base_train_time = time.time() - start_time

    # Evaluación del modelo
    y_pred_base = base_model.predict(X_test)
    base_f1 = f1_score(y_test, y_pred_base, average='macro')

    return base_model, base_f1, base_train_time, y_pred_base


def optimize_with_optuna(X_train, y_train, X_test, y_test, n_trials=20):
    """
    Realiza optimización de hiperparámetros para un modelo Random Forest usando Optuna.

    Busca maximizar el F1-score en el conjunto de prueba, evaluando distintos 
    conjuntos de parámetros mediante validación directa. Al finalizar, se 
    entrena un modelo final con los mejores parámetros encontrados.

    Args:
        X_train (np.ndarray): Conjunto de entrenamiento (features) escalado.
        y_train (np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray): Conjunto de prueba (features) escalado.
        y_test (np.ndarray): Etiquetas del conjunto de prueba.
        n_trials (int, opcional): Número de combinaciones a probar (default: 20).

    Returns:
        tuple:
            - optuna_best_model (RandomForestClassifier): Modelo final entrenado con los mejores parámetros.
            - optuna_f1 (float): F1-score del modelo optimizado en el test set.
            - optuna_time (float): Tiempo total de optimización en segundos.
            - optuna_results (pd.DataFrame): Detalles de todos los trials realizados.
            - y_pred_optuna (np.ndarray): Predicciones del modelo optimizado.
            - optuna_best_params (dict): Diccionario con los mejores hiperparámetros encontrados.
            - best_trial (int): Número del mejor trial según F1-score.
    """
    print(f"\nOptimizando modelo con Optuna ({n_trials} trials)")
    
    # Función objetivo para Optuna
    def objective(trial):
        # Espacio de búsqueda
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 200),
            'max_depth': trial.suggest_int('max_depth', 2, 32),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20)
        }
        
        # Crear y entrenar modelo
        model = RandomForestClassifier(**params, random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluar con F1-score
        y_pred = model.predict(X_test)
        return f1_score(y_test, y_pred, average='macro')

    # Ejecutar estudio de optimización
    print("Iniciando búsqueda de hiperparámetros con Optuna...", flush=True)
    start_time = time.time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    optuna_time = time.time() - start_time

    # Obtener mejores parámetros
    optuna_best_params = study.best_params
    optuna_best_model = RandomForestClassifier(**optuna_best_params, random_state=42)
    optuna_best_model.fit(X_train, y_train)
    y_pred_optuna = optuna_best_model.predict(X_test)
    optuna_f1 = f1_score(y_test, y_pred_optuna, average='macro')

    # Resultados de los trials
    optuna_results = study.trials_dataframe()
    
    return optuna_best_model, optuna_f1, optuna_time, optuna_results, y_pred_optuna, optuna_best_params, study.best_trial.number


def optimize_with_ray_tune(X_train, y_train, X_test, y_test, n_trials=20):
    """
    Realiza optimización de hiperparámetros para un modelo Random Forest utilizando Ray Tune.

    Usa búsqueda aleatoria (BasicVariantGenerator) sobre un espacio de hiperparámetros definido.
    Registra el F1-score de cada configuración y retorna el modelo con mejor desempeño, junto a
    sus resultados detallados. Incluye captura y despliegue de logs personalizados durante el proceso.

    Args:
        X_train (np.ndarray): Conjunto de entrenamiento (features) escalado.
        y_train (np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray): Conjunto de prueba (features) escalado.
        y_test (np.ndarray): Etiquetas del conjunto de prueba.
        n_trials (int, opcional): Número de configuraciones a evaluar (default: 20).

    Returns:
        tuple:
            - ray_best_model (RandomForestClassifier): Modelo final entrenado con los mejores parámetros.
            - ray_f1 (float): F1-score del modelo optimizado en el test set.
            - ray_time (float): Tiempo total de optimización en segundos.
            - ray_results_df (pd.DataFrame): DataFrame con el resumen de cada trial.
            - y_pred_ray (np.ndarray): Predicciones del modelo optimizado.
            - ray_best_params (dict): Hiperparámetros óptimos encontrados.
    """
  
    # Callback para mostrar progreso
    class ProgressCallback(Callback):
        def __init__(self, total_trials):
            self.total_trials = total_trials
            self.completed_trials = 0
            self.header_printed = False

        def on_trial_complete(self, iteration, trials, trial):
            if not self.header_printed:
                print("="*50)
                print(f"Optimizando modelo con Ray Tune ({self.total_trials} trials)")
                print("="*50)
                self.header_printed = True

            self.completed_trials += 1
            f1 = trial.last_result['f1_score']
            params = trial.config
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[I {timestamp}] Trial {self.completed_trials} finished with value: {f1:.4f} and parameters: {params}.", flush=True)

    # Configurar Ray
    ray.init(
        ignore_reinit_error=True,
        include_dashboard=False,
        logging_level=logging.CRITICAL,
        log_to_driver=False
    )

    # Función entrenable para Ray Tune
    def trainable(config):
        model = RandomForestClassifier(
            n_estimators=config["n_estimators"],
            max_depth=config["max_depth"],
            min_samples_split=config["min_samples_split"],
            random_state=42
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro')
        train.report({"f1_score": f1})

    # Espacio de búsqueda
    param_space = {
        "n_estimators": tune.randint(10, 200),
        "max_depth": tune.randint(2, 32),
        "min_samples_split": tune.randint(2, 20)
    }

    # Configurar el sintonizador
    tuner = Tuner(
        trainable,
        param_space=param_space,
        tune_config=tune.TuneConfig(
            metric="f1_score",
            mode="max",
            num_samples=n_trials,
            search_alg=BasicVariantGenerator(),
            max_concurrent_trials=4
        ),
        run_config=train.RunConfig(
            verbose=0,
            callbacks=[ProgressCallback(n_trials)]
        )
    )

    # Ejecutar optimización
    print("Iniciando búsqueda de hiperparámetros con Ray Tune...")
    start_time = time.time()
    results = tuner.fit()
    ray_time = time.time() - start_time

    # Obtener resultados
    best_result = results.get_best_result(metric="f1_score", mode="max")
    ray_best_params = best_result.config
    ray_f1 = best_result.metrics["f1_score"]

    # Convertir resultados a DataFrame
    ray_results = []
    for i, result in enumerate(results):
        ray_results.append({
            "trial": i + 1,
            "params": result.config,
            "f1_score": result.metrics["f1_score"]
        })
    ray_results_df = pd.DataFrame(ray_results)

    # Cerrar Ray
    ray.shutdown()

    # Entrenar modelo final con los mejores parámetros
    ray_best_model = RandomForestClassifier(
        n_estimators=ray_best_params["n_estimators"],
        max_depth=ray_best_params["max_depth"],
        min_samples_split=ray_best_params["min_samples_split"],
        random_state=42
    )
    ray_best_model.fit(X_train, y_train)
    y_pred_ray = ray_best_model.predict(X_test)

    return ray_best_model, ray_f1, ray_time, ray_results_df, y_pred_ray, ray_best_params

**Bloque 3:** Creacion de tablas y gráficos para comparación de modelos.

- **`compare_results()`** 
Compara los resultados de tres modelos (base, Optuna y Ray Tune) y genera visualizaciones, incluyendo tablas comparativas, gráficos de rendimiento, evolución de F1-Score y matrices de confusión.

In [ ]:
def compare_results(base_f1, base_time, optuna_f1, optuna_time, ray_f1, ray_time,
                   optuna_results, ray_results_df, y_test,
                   y_pred_base, y_pred_optuna, y_pred_ray, target_names):
    """
    Compara los resultados de tres modelos (base, Optuna y Ray Tune) y genera visualizaciones.

    Incluye:
    - Tabla comparativa de F1-score, tiempo de entrenamiento y número de trials.
    - Gráfico de barras con F1-score y tiempos.
    - Evolución del F1-score en los trials de Optuna y Ray Tune.
    - Matrices de confusión para cada modelo.

    Args:
        base_f1 (float): F1-score del modelo base.
        base_time (float): Tiempo de entrenamiento del modelo base.
        optuna_f1 (float): F1-score del modelo optimizado con Optuna.
        optuna_time (float): Tiempo total de optimización con Optuna.
        ray_f1 (float): F1-score del modelo optimizado con Ray Tune.
        ray_time (float): Tiempo total de optimización con Ray Tune.
        optuna_results (pd.DataFrame): Resultados de los trials de Optuna.
        ray_results_df (pd.DataFrame): Resultados de los trials de Ray Tune.
        y_test (np.ndarray): Etiquetas verdaderas del conjunto de prueba.
        y_pred_base (np.ndarray): Predicciones del modelo base.
        y_pred_optuna (np.ndarray): Predicciones del modelo optimizado con Optuna.
        y_pred_ray (np.ndarray): Predicciones del modelo optimizado con Ray Tune.
        target_names (list): Nombres de las clases objetivo.

    Returns:
        None. Solo imprime y visualiza resultados.
    """
    print("\n" + "="*50)
    print("5. COMPARACIÓN DE RESULTADOS")
    print("="*50)
    
    # Crear tabla comparativa
    comparison_data = {
        'Método': ['Modelo Base', 'Optuna', 'Ray Tune'],
        'F1-score': [base_f1, optuna_f1, ray_f1],
        'Tiempo (s)': [base_time, optuna_time, ray_time],
        'Número de Trials': ['-', len(optuna_results), len(ray_results_df)]
    }
    comparison_df = pd.DataFrame(comparison_data)

    # Mostrar tabla comparativa
    print("\nTabla Comparativa:")
    print("-"*60)
    print(comparison_df.to_string(index=False))
    print("-"*60)


    # Gráficos

    # Gráfico 1: Comparación de F1-scores
    fig1, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    sns.barplot(x='Método', y='F1-score', data=comparison_df, ax=axes[0], 
                hue='Método', palette="viridis", legend=False, dodge=False)
    axes[0].set_title('Comparación de Rendimiento (F1-score)')
    axes[0].set_ylim(0.9, 1.0)
    for i, v in enumerate(comparison_df['F1-score']):
        axes[0].text(i, v + 0.005, f"{v:.4f}", ha='center', fontsize=12)

    # Gráfico 2: Comparación de tiempos de ejecución
    sns.barplot(x='Método', y='Tiempo (s)', data=comparison_df, ax=axes[1], 
                hue='Método', palette="rocket", legend=False, dodge=False)
    axes[1].set_title('Comparación de Tiempo de Ejecución')
    for i, v in enumerate(comparison_df['Tiempo (s)']):
        axes[1].text(i, v + 0.1, f"{v:.2f}s", ha='center', fontsize=12)

    # Guardar las figuras
    fig1.savefig("comparacion_metricas.png", dpi=300, bbox_inches='tight')
    plt.show()

def plot_top_features(models, feature_names, model_names, top_n=5):
    """
    Genera una figura con los gráficos de barras verticales del top N de variables más importantes
    para cada modelo (por importancia de Gini).

    Args:
        models (list): Lista de modelos RandomForest entrenados (base, Optuna, Ray).
        feature_names (list): Lista de nombres de características.
        model_names (list): Nombres de los modelos correspondientes.
        top_n (int): Número de variables top a mostrar.

    Returns:
        None. Muestra y guarda la figura.
    """
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))  # 1 fila, 3 columnas
    fig.suptitle("Top 5 variables más importantes por modelo", fontsize=16)

    for i, (model, name) in enumerate(zip(models, model_names)):
        importances = model.feature_importances_
        top_indices = np.argsort(importances)[-top_n:][::-1]
        top_features = [feature_names[j] for j in top_indices]
        top_values = importances[top_indices]

        sns.barplot(x=top_features, y=top_values, ax=axes[i], hue=top_features, palette="viridis", legend=False)
        axes[i].set_title(f"{name}", fontsize=14)
        axes[i].set_xlabel("Variable")
        axes[i].set_ylabel("Importancia")
        axes[i].set_ylim(0, max(top_values)*1.1)  # un poco de margen arriba

        # Agregar etiquetas con valores encima de las barras
        for j, v in enumerate(top_values):
            axes[i].text(j, v + 0.005, f"{v:.3f}", ha='center', fontsize=10)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig("top_features_importancia.png", dpi=300, bbox_inches='tight')
    plt.show()

**Bloque 4:** Funciones de ejecución.

Debido a que Ray Tune produce errores como borrar las celdas de salida previas a us ejecución, la función main fue segmentada en dos partes:

- **`main()`**
Función que ejecuta el flujo: carga de datos, entrenamiento, creación de modelo base y optimizados con Optuna y Ray Tune.

- **`main_2()`**
Función que toma los resultados obtenidos por main() y muestra los resultados obtenidos a partir de la comparación de los tres modelos.

In [ ]:
def main():
    """
    Ejecuta el flujo completo de entrenamiento y optimización del modelo.

    Incluye:
    - Configuración del entorno y estilos.
    - Carga y preprocesamiento del dataset de cáncer de mama.
    - Entrenamiento de un modelo base sin tuning.
    - Optimización de hiperparámetros con Optuna y Ray Tune.
    - Evaluación de desempeño (F1-score y tiempo).

    Returns:
        tuple: Variables necesarias para reportes y comparación posterior. Incluye:
            - info_df (pd.DataFrame): Resumen del dataset.
            - optuna_best_params (dict): Mejores parámetros de Optuna.
            - ray_best_params (dict): Mejores parámetros de Ray Tune.
            - base_f1 (float): F1-score del modelo base.
            - base_time (float): Tiempo de entrenamiento del modelo base.
            - optuna_f1 (float): F1-score con Optuna.
            - optuna_time (float): Tiempo de optimización con Optuna.
            - best_trial (int): ID del mejor trial de Optuna.
            - ray_f1 (float): F1-score con Ray Tune.
            - ray_time (float): Tiempo de optimización con Ray Tune.
            - optuna_results (pd.DataFrame): Resultados de Optuna.
            - ray_results (pd.DataFrame): Resultados de Ray Tune.
            - y_test (np.ndarray): Etiquetas reales del test.
            - y_pred_base (np.ndarray): Predicciones del modelo base.
            - y_pred_optuna (np.ndarray): Predicciones del modelo optimizado con Optuna.
            - y_pred_ray (np.ndarray): Predicciones del modelo optimizado con Ray Tune.
            - target_names (np.ndarray): Nombres de las clases objetivo.
    """
    # 0. Configuración inicial
    setup_environment()
    
    # 1. Carga y preprocesamiento de datos
    train = pd.read_csv('Training.csv')
    test = pd.read_csv('Testing.csv')

    # Combinar datasets (concatenar filas)
    data = pd.concat([train, test], ignore_index=True)

    # Limpiar columna 'Unnamed: 133' si existe
    if 'Unnamed: 133' in data.columns:
        data = data.drop(columns=['Unnamed: 133'])
        print("\nColumna 'Unnamed: 133' eliminada.")

    # Borrar duplicados
    data = data.drop_duplicates()

    X_train, X_test, y_train, y_test, data, info_df = load_and_preprocess_data(data)
    target_names = sorted(data['prognosis'].unique())
    
    # 2. Modelo base (sin tuning)
    base_model, base_f1, base_time, y_pred_base = train_baseline_model(
        X_train, y_train, X_test, y_test, target_names
    )

    # 3. Optimización con Ray Tune (20 trials). Se ejecuta primero para evitar problemas de memoria con Optuna.
    ray_model, ray_f1, ray_time, ray_results, y_pred_ray, ray_best_params = optimize_with_ray_tune(
        X_train, y_train, X_test, y_test, n_trials=20
    )
    
    # 4. Optimización con Optuna (20 trials)
    optuna_model, optuna_f1, optuna_time, optuna_results, y_pred_optuna, optuna_best_params, best_trial = optimize_with_optuna(
        X_train, y_train, X_test, y_test, n_trials=20
    )

    # Mensajes finales antes de retornar
    print("\nDatos cargados y preprocesados correctamente.")
    print(f"Modelo base entrenado sin optimización en {base_time:.2f} segundos.")
    print(f"Modelo optimizado con Optuna completado en {optuna_time:.2f} segundos.")
    print(f"Modelo optimizado con Ray Tune completado en {ray_time:.2f} segundos.")

    return (info_df, optuna_best_params, ray_best_params, base_f1, base_time,
           optuna_f1, optuna_time, best_trial, ray_f1, ray_time,
           optuna_results, ray_results, y_test,
           y_pred_base, y_pred_optuna, y_pred_ray, target_names,
           base_model, optuna_model, ray_model, data)        

def main_2(info_df, optuna_best_params, ray_best_params, base_f1, base_time,
           optuna_f1, optuna_time, best_trial, ray_f1, ray_time,
           optuna_results, ray_results, y_test,
           y_pred_base, y_pred_optuna, y_pred_ray, target_names,
           base_model, optuna_model, ray_model, data):
    """
    Segunda parte del flujo: resumen de resultados, comparación y visualización.

    Esta función:
    - Muestra resumen del dataset.
    - Imprime mejores hiperparámetros encontrados.
    - Reporta desempeño en F1-score y tiempo de entrenamiento.
    - Compara gráficamente los métodos (modelo base, Optuna y Ray Tune).
    - Guarda los resultados finales en un archivo CSV.

    Args:
        info_df (pd.DataFrame): Resumen del dataset.
        optuna_best_params (dict): Mejores parámetros de Optuna.
        ray_best_params (dict): Mejores parámetros de Ray Tune.
        base_f1 (float): F1-score del modelo base.
        base_time (float): Tiempo de entrenamiento del modelo base.
        optuna_f1 (float): F1-score del modelo optimizado con Optuna.
        optuna_time (float): Tiempo total de optimización con Optuna.
        best_trial (int): Trial con mejor resultado en Optuna.
        ray_f1 (float): F1-score del modelo optimizado con Ray Tune.
        ray_time (float): Tiempo total de optimización con Ray Tune.
        optuna_results (pd.DataFrame): Resultados de los trials de Optuna.
        ray_results (pd.DataFrame): Resultados de los trials de Ray Tune.
        y_test (np.ndarray): Etiquetas reales del conjunto de prueba.
        y_pred_base (np.ndarray): Predicciones del modelo base.
        y_pred_optuna (np.ndarray): Predicciones del modelo Optuna.
        y_pred_ray (np.ndarray): Predicciones del modelo Ray Tune.
        target_names (np.ndarray): Nombres de las clases objetivo.

    Returns:
        None. Imprime, visualiza y guarda resultados.
    """
    # Mostrar info del dataset en tabla
    print("Resumen del dataset:")
    info_df = info_df.T  # Transpone el DataFrame
    info_df.columns = ["Valor"]  # Renombra la única columna resultante
    print(info_df)

    # Mostrar mejores parámetros
    print("\nMejores parámetros encontrados por Optuna:")
    for param, value in optuna_best_params.items():
        print(f"- {param}: {value}")

    print("\nMejores parámetros encontrados por Ray Tune:")
    for param, value in ray_best_params.items():
        print(f"- {param}: {value}")

    # Mostrar resultados de cada modelo
    print("\nResultados del modelo base:")
    print(f"- F1-score: {base_f1:.4f}")
    print(f"- Tiempo de entrenamiento: {base_time:.2f} segundos")
    
    print("\nResultados del modelo optimizado con Optuna:")
    print(f"- F1-score: {optuna_f1:.4f}")
    print(f"- Tiempo total de optimización: {optuna_time:.2f} segundos")
    print(f"- Mejor trial: #{best_trial}")

    print("\nResultados del modelo optimizado con Ray Tune:")
    print(f"- F1-score: {ray_f1:.4f}")
    print(f"- Tiempo total de optimización: {ray_time:.2f} segundos")

    # Comparación de hiperparámetros
    param_comparison = pd.DataFrame({
        'Parámetro': list(optuna_best_params.keys()),
        'Optuna': list(optuna_best_params.values()),
        'Ray Tune': [ray_best_params[k] for k in optuna_best_params.keys()]
    })

    print("\nComparación de Hiperparámetros:")
    print(param_comparison.to_string(index=False))

    mejor_metodo = 'Optuna' if optuna_f1 >= ray_f1 and optuna_f1 >= base_f1 else 'Ray Tune' if ray_f1 > base_f1 else 'Modelo Base'
    print(f"\nMejor modelo: {mejor_metodo} (F1-score: {max(base_f1, optuna_f1, ray_f1):.4f})")


    # 5. Comparación de resultados (con parámetros adicionales)
    compare_results(
        base_f1, base_time, 
        optuna_f1, optuna_time, 
        ray_f1, ray_time,
        optuna_results, 
        ray_results,
        y_test,
        y_pred_base,
        y_pred_optuna,
        y_pred_ray,
        target_names
    )

    feature_names = data.columns.drop('prognosis')  # Obtener nombres de features
    plot_top_features(
        models=[base_model, optuna_model, ray_model],
        feature_names=feature_names,
        model_names=["Modelo Base", "Optuna", "Ray Tune"]
    )

# 4. Visualización de resultados

Para evitar problemas en las celdas de salida, la ejecución de realiza en dos pasos, en dos celdas de codigo distintas.

---

- **`Primer paso`**
Almacenar en una variable todos los resultados generados por los modelos a comparar.

- **`Segundo paso`**
Usar los resultados obtenidos con main() para mostrarlos junto a las comparaciones de modelos.

In [ ]:
# Primer paso
resultados = main()

In [ ]:
# Segundo paso
main_2(*resultados)

## Análisis de Resultados y Reflexiones Finales

### Justificaciones: 
El dataset utilizado presentó algunos problemas que fueron solucionados según lo siguiente:
    - El dataset venía separado en Training.csv y Testing.cvs, pero al revisarlos, el split estaba hecho en una relacion similar a 99.15-0.85, por lo que el test era muy pequeño. De esta manera se juntaron ambos .csv mediante concat.
    - Dentro de las columnas, había una llamada **Unnamed: 133**, la cual se eliminó del dataset debido a que no presentaba valores.
    - El dataset presentaba datos duplicados, y al hcer el analisis exploratorio, se observó que la mayoria del dataset era duplicado, por lo que se eliminaron mediante drop_duplicates(), quedando en poco mas de 300 filas.

### 1. Comparación del rendimiento del modelo optimizado vs modelo base

Todos los modelos evaluados —base, optimizado con Optuna y optimizado con Ray Tune— alcanzaron un **F1-score perfecto de 1.0000**, lo que sugiere que el problema puede ser relativamente sencillo o que las clases están muy bien separadas en el espacio de características. 

Junto a esto es importante mencionar que desde la fuente del dataset, muchas personas han reportado que los datos siempre arrojan un F1-Score de 1, independiente del modelo que usen, aun sin optimizar, por lo que hay que tomar esto en cuenta a la hora de intentar interpretar los resultados, ya que el dataset puede haber sido creado con la finalidad de crear datos de practica para codigo mas que para evaluar modelos.

Sin embargo, al comparar los **tiempos de ejecución**, se observan diferencias relevantes:

| Método        | F1-score | Tiempo (s) |
|---------------|----------|------------|
| Modelo Base   | 1.0000   | 0.09       |
| Optuna        | 1.0000   | 2.19       |
| Ray Tune      | 1.0000   | 12.34      |

- El **modelo base** ofrece un rendimiento perfecto en **solo 0.09 segundos**, sin necesidad de optimización.
- **Optuna** logra el mismo rendimiento en un tiempo aceptable (2.19s) y puede ser útil para ajustar el modelo a nuevos datos.
- **Ray Tune**, aunque potente y paralelizable, requiere más tiempo (12.34s) para obtener resultados similares.

> **Conclusión**: En este caso particular, el modelo base es suficiente. Sin embargo, la optimización puede ser valiosa en problemas más complejos o con ruido.

---

### 2. Importancia de las variables e interpretabilidad

En este caso, las variables mas importantes fueron graficadas, y dentro de eso, en el modelo base y el optimizado con Ray Tune la mas importante fue (al menos en esta ejecución) **itching** o picazón, y tiene sentido, ya que es un síntoma común en muchas enfermedades, especialmente en enfermedades de la piel, alergias, infecciones, o incluso algunas enfermedades sistémicas que pueden manifestarse con síntomas dermatológicos. Por lo tanto, puede ser un predictor fuerte para distinguir enfermedades relacionadas.

Por el lado del modelo optimizado con Optuna, la mas importante fue **muscle_pain** o dolor muscular, es otro síntoma muy frecuente que aparece en muchas condiciones infecciosas, inflamatorias, virales (como gripe o dengue) o autoinmunes, y puede ayudar a diferenciar ciertas enfermedades.

En problemas de clasificación multiclase con muchos atributos (en este caso, 132 variables predictoras), entender la importancia de las variables es fundamental para:

- Interpretar el comportamiento del modelo.
- Identificar los atributos más influyentes en la predicción.
- Reducir la dimensionalidad, si fuese necesario, eliminando variables irrelevantes.
- Guiar futuras decisiones sobre recolección de datos o ingeniería de características.

La importancia de una variable nos indica cuánto contribuye esa característica a la precisión del modelo. En modelos como RandomForestClassifier, se calcula con base en la mejora del criterio de división (por ejemplo, Gini o Entropía) al usar esa variable en los árboles de decisión.

Aunque los modelos basados en árboles como Random Forest no son tan interpretables como modelos lineales simples, permiten estimar:

- Qué variables son más relevantes para el modelo.
- Qué características ayudan a distinguir entre múltiples clases.
- Cómo podrían reaccionar las predicciones si ciertos atributos cambian.

---

### 3. Ventajas y desventajas de Optuna vs Ray Tune

| Criterio              | Optuna                        | Ray Tune                       |
|-----------------------|-------------------------------|--------------------------------|
| Facilidad de uso      | Muy simple e intuitivo        | Más complejo de configurar     |
| Velocidad             | Rápido                        | Más lento (por overhead)       |
| Paralelismo           | Limitado                      | Excelente soporte              |
| Visualización         | Básica                        | Avanzada (si se habilita)      |
| Integración           | Ligero, fácil de integrar     | Requiere más setup             |

#### Ventajas de Optuna:
- Sencillo de implementar, ideal para notebooks y proyectos rápidos.
- Eficiente para problemas de baja a media complejidad.
- Espacio de búsqueda flexible y bien documentado.

#### Ventajas de Ray Tune:
- Excelente para escalamiento en entornos distribuidos o multi-GPU.
- Soporte avanzado para callbacks, dashboards y logging.
- Integración con frameworks como PyTorch, XGBoost, LightGBM, entre otros.

> **Recomendación**: Utilizar **Optuna** para entornos locales o tareas exploratorias, y **Ray Tune** cuando se requiera optimización a gran escala o integración con recursos distribuidos.

---

### Conclusión general

Aunque el rendimiento no varió entre modelos en este caso, este experimento permitió:

- Validar el poder predictivo del modelo base.
- Evaluar herramientas de optimización automatizada.
- Comprender mejor las variables más relevantes para el diagnóstico.
- Reflexionar sobre cuándo y por qué optimizar hiperparámetros.

> En contextos más complejos, desequilibrados o con datasets ruidosos, la optimización puede marcar una diferencia significativa en el rendimiento final del modelo.